Question 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
df_train = pd.read_csv('Dry_Bean_train.csv') 
df_x_train = df_train.drop("Class", axis=1) 
df_y_train = df_train['Class']
x_train = df_x_train.to_numpy()

df_test = pd.read_csv('Dry_Bean_test.csv') 
df_x_test = df_test.drop("Class", axis=1)
df_y_test = df_test['Class']
x_test = df_x_test.to_numpy()

encoder = preprocessing.LabelEncoder()
y_train_enc = encoder.fit_transform(df_y_train)
y_test_enc = encoder.fit_transform(df_y_test)

In [ ]:
def train(X, y, epochs=100, lr=1):
    d , c = x_train.shape[1] , max(y_train_enc)+1
    w = np.ones((c, d+1))
    w_opt = np.ones((c, d+1))
    num_samples = X.shape[0]
    itr = 0
    J_min = 100000000
    for epoch in range(epochs):
        temp = np.c_[X,y]
        np.random.shuffle(temp)
        X = temp[:, :17]
        y = temp[:, -1]
        for i in range(num_samples):
            x = X[i]
            true_label = y[i]
            scores = np.dot(w, x.T)
            predicted_label = np.argmax(scores)
            if predicted_label != true_label:
                w[int(true_label)] += lr * x
                w[predicted_label] -= lr * x
            itr = itr +1

            if itr>1224799:
                jw = Jw(X, w[int(true_label)], w[predicted_label])
                if jw<J_min:
                    J_min = jw
                    w_opt = w
    return w, J_min, w_opt

def Jw(X, wk, wl):
    j=0
    for i in range(X.shape[0]):
        j = j + np.dot(wk,X[i].T) - np.dot(wl,X[i].T)
    return -j

def predict(X, w):
    scores = np.dot(w, X.T)
    return np.argmax(scores, axis=0)

def accuracy(y_true, y_pred):
    score = np.equal(y_true, y_pred)
    return np.sum(score), np.mean(score)

In [ ]:
std = StandardScaler()

std.fit(x_train)

x_train_normalized = std.transform(x_train)

x_train_normalized_aug = np.hstack((np.ones((x_train_normalized.shape[0], 1)), x_train_normalized))

w, J_min, w_opt = train(x_train_normalized_aug, y_train_enc)

y_train_pred = predict(x_train_normalized_aug, w_opt)

train_m_points, train_acc = accuracy(y_train_enc, y_train_pred)

print("Train Data Accuracy ", train_acc*100, "%")

x_test_normalized = std.transform(x_test)

x_test_normalized_aug = np.hstack((np.ones((x_test_normalized.shape[0], 1)), x_test_normalized))

y_test_pred = predict(x_test_normalized_aug, w_opt)

test_m_points, test_acc = accuracy(y_test_enc, y_test_pred)

print("Test Data Accuracy ", test_acc*100, "%")

cm = confusion_matrix(y_test_enc, y_test_pred) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm) 
disp.plot() 
plt.figure() 
plt.show()

In [ ]:
for i in range(7):
    print("Class", i+1, "Weight Vector: " , w_opt[i])
    print("Class", i+1, "Weight Magnitude: ", np.linalg.norm(w_opt[i]))
    print()

In [ ]:
train_mean_list = []
test_mean_list = []
w_dict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
w_dict_list = {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: []}
cm_list = []

for i in range(10):
    w, J_min, w_opt = train(x_train_normalized_aug, y_train_enc)
    y_train_pred = predict(x_train_normalized_aug, w_opt)
    train_m_points, train_acc = accuracy(y_train_enc, y_train_pred)
    train_mean_list.append(train_acc)

    y_test_pred = predict(x_test_normalized_aug, w_opt)
    test_m_points, test_acc = accuracy(y_test_enc, y_test_pred)
    test_mean_list.append(test_acc)

    cm_list.append(confusion_matrix(y_test_enc, y_test_pred))

    for i in range(7):
        mag = np.linalg.norm(w_opt[i])
        w_dict[i] += mag
        w_dict_list[i].append(mag)

In [ ]:
print("Mean of train data accuracy: ", np.mean(train_mean_list)*100, "%")
print("Mean of test data accuracy: ", np.mean(test_mean_list)*100, "%")
print("Standard deviation of train data accuracy: ", np.std(train_mean_list)*100, "%")
print("Standard deviation of test data accuracy: ", np.std(test_mean_list)*100, "%")

In [ ]:
for i in range(7):
    print("Class ", i+1, " Weight Magnitude Mean: ", w_dict[i]/10)
    print("Class ", i+1, " Weight Magnitude Standard Deviation: ", np.std(w_dict_list[i]))
    print()

In [ ]:
print("Confussion matrix mean: ")
print(np.mean(cm_list, axis=0))

In [ ]:
print("Confussion matrix standard deviation: ")
print(np.std(cm_list, axis=0))